In [86]:
# General
import numpy as np
import pickle as pk
import math

# For optimization
import pygmo as pg

# For computing acceleration and potential
import polyhedral_gravity as model

# For working with the mesh
import mesh_utility
import tetgen
import meshio as mio
import openmesh as om

# For Plotting
import pyvista as pv
import mesh_plotting
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from matplotlib import cm


class basic_udp:
    
    def fitness(self,x):
        print("fitness call")
        # Initial information
        r = np.transpose(x[0:3]) # Start Position
        v = np.transpose(x[3:6]) # Initial velocity

        # Creating the mesh (TetGen)
        mesh, vertices, faces = mesh_utility.create_mesh()

        # Defining additional parameters
        density = 533           # https://sci.esa.int/web/rosetta/-/14615-comet-67p
        target_altitude = 10    # In unit: [km]
        t_end = 1000            # Final time
        t_0 = 0                 # Starting time
        d_t = 0.1               # Number of steps along orbit 

        # Array containing times for summation
        time_list = np.arange(t_0,t_end,d_t)

        # Fitness value (to be maximized)
        fit_val = 0

        
        for t in time_list:

            # Retrieve information at current position
            altitude = compute_Altitude(r, mesh)
            potential, a, tensor = model.evaluate(vertices, faces, density, r)
            a = np.array(a)

            # Computing velocity and position for next time-step
            
            ##################
            #print("Value of v: ", v, type(v), ",   Value of a: ", a, type(a), ",    Value of r: ", r, type(r), ",    Value of d_t: ", d_t, type(d_t))
            ##################

            v_n = v + d_t * a
            r_n = r + d_t * v

            # Evaluate current position
            fit_val += abs(target_altitude - altitude)

            # Update current velocity and position
            v = v_n
            r = r_n


    def get_bounds(self):
        # Bounds for the initial domain (i.e bounds for possible x-chromosomes)
        #   min=[r_x,r_y,r_z,v_x,v_y,v_z], max=[r_x,r_y,r_z,v_x,v_y,v_z]
        # Good starting boundaries could e.g be right outside the most protruding part of the the mesh 
        return ([10,5,5,1000,1000,1000],[40,20,20,8000,8000,8000])



def compute_Altitude(r,mesh):
    # Obtaining closest distance from mesh to satelite: 
    #    See PyVista function: https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.find_closest_point.html
    index = mesh.grid.find_closest_point(r) 
    closest_point = mesh.grid.points[index]
    altitude = np.linalg.norm(closest_point-r)
    return altitude



def get_trajectory(mesh, vertices, faces, x,t_end):

    #to be finished.

    # Initial information
    r = np.transpose(x[0:3]) # Start Position
    v = np.transpose(x[3:6]) # Initial velocity

    # Defining additional parameters
    density = 533           # https://sci.esa.int/web/rosetta/-/14615-comet-67p
    target_altitude = 10    # In unit: [km]
    t_end = 1000            # Final time
    t_0 = 0                 # Starting time
    d_t = 1                 # Number of steps along orbit 

    time_list = np.arange(t_0,t_end,d_t)
    
    # Numpy Arrays to store trajectory information
    r_store = np.zeros()
    v_store = np.zeros()

    # Add starting position to memory
    r_store[:,1] = r
    v_store[:,1] = v
    i = 2

    for t in time_list:

        # Retrieve information at current position
        altitude = compute_Altitude(r, mesh)
        potential, a, tensor = model.evaluate(vertices, faces, density, r)
        a = np.array(a)

        # Computing velocity and position for next time-step
        v_n = v + d_t * a
        r_n = r + d_t * v

        # Update current velocity and position
        v = v_n
        r = r_n

        # Storing updated trajectory information
        r_store[:,i] = r
        v_store[:,i] = v
        i += 1

    return r_store, v_store



def plot_trajectory(mesh, vertices, faces, r_store):

    # Plotting mesh of asteroid/comet
    mesh_plot = pv.Plotter()
    mesh_plot.add_mesh(mesh.grid, show_edges=True)
    mesh_plot.show_bounds(grid='front',location='outer',all_edges=True)

    # Plotting trajectory
    trajectory_plot = pv.PolyData(r_store)
    mesh_plot.add_mesh(trajectory_plot, color=[0.6, 0.2, 0.1])


def plot_evolutions():
    ...




In [87]:

# Defining and executing optimization
prob = pg.problem(basic_udp())
print(prob)

Problem name: <class '__main__.basic_udp'>
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [10, 5, 5, 1000, 1000, ... ]
	Upper bounds: [40, 20, 20, 8000, 8000, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



In [88]:
algo = pg.algorithm(pg.de(gen = 20))
pop = pg.population(prob = basic_udp(), size = 1)

fitness call
Reading file...
Physical dimension along x (UN-normalized):  5.0025703125 Km
Tetrahralize...
Writing vtk files...
Writing polyhedron-cpp input files..
Writing FORTRAN files..
Writing other files...
Finished.
Value of v:  [1130.28121752 2284.09983929 3090.02345037] <class 'numpy.ndarray'> ,   Value of a:  [7.63811499e-10 4.90048891e-10 5.02218120e-10] <class 'numpy.ndarray'> ,    Value of r:  [30.10715581 19.36822436 19.78819366] <class 'numpy.ndarray'> ,    Value of d_t:  0.1 <class 'float'>
Value of v:  [1130.28121752 2284.09983929 3090.02345037] <class 'numpy.ndarray'> ,   Value of a:  [3.00064366e-12 5.19422086e-12 6.89269162e-12] <class 'numpy.ndarray'> ,    Value of r:  [143.13527756 247.77820829 328.79053869] <class 'numpy.ndarray'> ,    Value of d_t:  0.1 <class 'float'>
Value of v:  [1130.28121752 2284.09983929 3090.02345037] <class 'numpy.ndarray'> ,   Value of a:  [7.60681175e-13 1.41403988e-12 1.89393703e-12] <class 'numpy.ndarray'> ,    Value of r:  [256.163399

ValueError: array has incorrect number of dimensions: 0; expected 1

In [ ]:
pop = algo.evolve(pop)

In [4]:
print(os.getcwd())

/Users/rasmus/Master Thesis/My Repositories/67P-Swarm/Introductory Tests


In [ ]:
uda = algo.extract(pg.de)
uda.get_log() 
print(pop.champion_f) 


In [ ]:

t_end = 100
x = ...
# Creating the mesh and plotting results
mesh, vertices, faces = mesh_utility.create_mesh()

r_store, v_store = get_trajectory(mesh, vertices, faces, x,t_end)

plot_trajectory(mesh, vertices, faces, r_store)

In [ ]:
    #altitude = math.dist(closest_point, r)
